In [1]:
!pip install datasets nltk spacy tensorflow pandas tqdm --quiet

In [2]:
import nltk
import spacy

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Download the required spaCy model
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
import pandas as pd
from datasets import load_dataset
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from tqdm import tqdm  # Progress bar for processing

c:\Users\madhe\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# pre processing

In [5]:

dataset = load_dataset("sentence-transformers/wikihow")
df = pd.DataFrame(dataset["train"])  # Convert to Pandas DataFrame

# Check first few rows
print(df.head())


                                                text  \
0  So you're a new or aspiring artist and your cr...   
1  If you want to be well-read, then, in the word...   
2  Stage names are used by all types of performer...   
3  Burlesque in the U.S. is one of the few art fo...   
4  This article describes a general idea on how t...   

                                      summary  
0                        Sell Fine Art Online  
1                                Be Well Read  
2                           Pick a Stage Name  
3                     Get More Burlesque Gigs  
4  Get a Record Deal With Phantom City Studio  


In [ ]:
df.columns

In [ ]:

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Common abbreviations that should be preserved
ABBREVIATIONS = {"u.s.", "u.k.", "m.p.h.", "dr.", "mr.", "ms.", "jr.", "sr.", "ph.d."}

def clean_text(text):
    if not isinstance(text, str):  # Handle missing values
        return ""

    # Lowercasing
    text = text.lower()

    # Sentence segmentation
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    cleaned_sentences = []

    for sent in sentences:
        words = word_tokenize(sent)  # Tokenization

        processed_words = []
        for word in words:
            if word in ABBREVIATIONS:  # Preserve abbreviations
                processed_words.append(word)
            elif word.isalnum() and word not in stop_words:  # Remove stopwords & special characters
                processed_words.append(lemmatizer.lemmatize(word))

        cleaned_sentences.append(" ".join(processed_words))

    # Reconstruct text from cleaned sentences
    return " ".join(cleaned_sentences)


'\n# Load spaCy model\nnlp = spacy.load("en_core_web_sm")\n\n# Initialize lemmatizer and stopwords\nlemmatizer = WordNetLemmatizer()\nstop_words = set(stopwords.words("english"))\n\n# Common abbreviations that should be preserved\nABBREVIATIONS = {"u.s.", "u.k.", "m.p.h.", "dr.", "mr.", "ms.", "jr.", "sr.", "ph.d."}\n\ndef clean_text(text):\n    if not isinstance(text, str):  # Handle missing values\n        return ""\n\n    # Lowercasing\n    text = text.lower()\n\n    # Sentence segmentation\n    doc = nlp(text)\n    sentences = [sent.text for sent in doc.sents]\n\n    cleaned_sentences = []\n\n    for sent in sentences:\n        words = word_tokenize(sent)  # Tokenization\n\n        processed_words = []\n        for word in words:\n            if word in ABBREVIATIONS:  # Preserve abbreviations\n                processed_words.append(word)\n            elif word.isalnum() and word not in stop_words:  # Remove stopwords & special characters\n                processed_words.append(lem

In [ ]:

# Apply cleaning function to both text and summary
df['cleaned_text'] = df['text'].apply(clean_text)
df['cleaned_summary'] = df['summary'].apply(clean_text)

print(df[['cleaned_text', 'cleaned_summary']].head())

"\n# Apply cleaning function to both text and summary\ndf['cleaned_text'] = df['text'].apply(clean_text)\ndf['cleaned_summary'] = df['summary'].apply(clean_text)\n\nprint(df[['cleaned_text', 'cleaned_summary']].head())\n"

In [ ]:

import re
def clean_special_chars(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    return text
df['cleaned_text'] = df['cleaned_text'].apply(clean_special_chars)
df['cleaned_summary'] = df['cleaned_summary'].apply(clean_special_chars)


"\nimport re\ndef clean_special_chars(text):\n    text = re.sub(r'http\\S+', '', text)  # Remove URLs\n    text = re.sub(r'\\S+@\\S+', '', text)  # Remove emails\n    text = re.sub(r'[^A-Za-z0-9\\s]', '', text)  # Remove special characters\n    return text\ndf['cleaned_text'] = df['cleaned_text'].apply(clean_special_chars)\ndf['cleaned_summary'] = df['cleaned_summary'].apply(clean_special_chars)\n"

In [ ]:

df['cleaned_text'] = df['cleaned_text'].str.replace(r'\s+', ' ', regex=True).str.strip()
df['cleaned_summary'] = df['cleaned_summary'].str.replace(r'\s+', ' ', regex=True).str.strip()


"\ndf['cleaned_text'] = df['cleaned_text'].str.replace(r'\\s+', ' ', regex=True).str.strip()\ndf['cleaned_summary'] = df['cleaned_summary'].str.replace(r'\\s+', ' ', regex=True).str.strip()\n"

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_text_sequences(sequences, max_len=None):
    """
    not using now, will use it for training lstm/gru
    Pads the sequences to the same length.

    Parameters:
    - sequences (list of list of int): List of tokenized sentences (or texts).
    - max_len (int, optional): Maximum length of sequences. If None, it will pad to the length of the longest sequence.

    Returns:
    - padded_sequences (numpy array): Padded sequences.
    """

    # Pad the sequences
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

    return padded_sequences
  

'\nfrom tensorflow.keras.preprocessing.sequence import pad_sequences\n\ndef pad_text_sequences(sequences, max_len=None):\n    """\n    not using now, will use it for training lstm/gru\n    Pads the sequences to the same length.\n\n    Parameters:\n    - sequences (list of list of int): List of tokenized sentences (or texts).\n    - max_len (int, optional): Maximum length of sequences. If None, it will pad to the length of the longest sequence.\n\n    Returns:\n    - padded_sequences (numpy array): Padded sequences.\n    """\n\n    # Pad the sequences\n    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding=\'post\', truncating=\'post\')\n\n    return padded_sequences\n    '

In [ ]:
df.to_csv("preprocessed_data.csv", index=False)

# Retrieval tasks

In [ ]:
import pandas as pd
import numpy as np

# Load the preprocessed data
file_path = "preprocessed_data.csv"  # Ensure this is the correct path
df = pd.read_csv(file_path)

# Fill NaN values with an empty string
df['cleaned_text'] = df['cleaned_text'].fillna("")
df['cleaned_summary'] = df['cleaned_summary'].fillna("")

# Combine both columns to create a corpus
corpus = df['cleaned_text'].tolist() + df['cleaned_summary'].tolist()

# Remove empty strings if necessary
corpus = [text for text in corpus if text.strip()]

print(f"Corpus size: {len(corpus)}")


Corpus size: 257073


In [17]:
len(corpus)

257073

In [18]:
query = "how to craft an engaging science fiction plot twist"

# tf-idf

In [22]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def jaccard_similarity(query, document):
    set_query = set(query.split())
    set_doc = set(document.split())
    intersection = len(set_query & set_doc)
    union = len(set_query | set_doc)
    return intersection / union if union != 0 else 0

def mmr(query_tfidf, tfidf_matrix, lambda_param=0.5, top_k=5):
    cosine_sim = cosine_similarity(query_tfidf, tfidf_matrix).flatten()  # Keep sparse format
    
    selected_docs = []
    candidate_docs = list(range(len(cosine_sim)))
    mmr_scores = []

    for _ in range(top_k):
        if not selected_docs:
            best_idx = np.argmax(cosine_sim)
        else:
            max_sim_to_selected = np.max(cosine_similarity(tfidf_matrix[candidate_docs], tfidf_matrix[selected_docs]), axis=1)
            marginal_relevance = lambda_param * cosine_sim[candidate_docs] - (1 - lambda_param) * max_sim_to_selected
            best_idx = candidate_docs[np.argmax(marginal_relevance)]
        
        selected_docs.append(best_idx)
        candidate_docs.remove(best_idx)
        mmr_scores.append(cosine_sim[best_idx])  

    return selected_docs, mmr_scores

def compute_ndcg(relevance_scores, k):
    # Sorting the relevance scores to simulate ground truth
    sorted_relevance = sorted(relevance_scores, reverse=True)
    
    # Compute DCG
    dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores[:k])])
    
    # Compute ideal DCG (IDCG) using sorted relevance
    idcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(sorted_relevance[:k])])
    
    # Avoid division by zero
    ndcg = dcg / idcg if idcg != 0 else 0
    return ndcg

def tfidf_retrieval(query, corpus, top_k=5):
    vectorizer = TfidfVectorizer(dtype=np.float32, max_features=20000)  # Reduce memory
    tfidf_matrix = vectorizer.fit_transform(corpus)
    query_tfidf = vectorizer.transform([query])  # Keep sparse

    # Compute MMR
    mmr_indices, mmr_scores = mmr(query_tfidf, tfidf_matrix, top_k=top_k)

    # Compute Jaccard similarity
    jaccard_scores = [jaccard_similarity(query, corpus[i]) for i in mmr_indices]

    # Compute NDCG (using the cosine similarity scores as relevance)
    ndcg_score = compute_ndcg(mmr_scores, top_k)

    # Store results
    retrieved_docs = [(corpus[i], mmr_scores[j], jaccard_scores[j]) for j, i in enumerate(mmr_indices)]

    return retrieved_docs, ndcg_score


retrieved_docs, ndcg_score = tfidf_retrieval(query, corpus, top_k=5)

# Printing results with MMR scores and NDCG
print(f"TF-IDF Retrieval - Query: {query}")
for idx, (retrieved_text, cos_sim, jaccard_sim) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}:")
    print(f"Text: {retrieved_text}")
    print(f"Cosine Similarity Score: {cos_sim:.4f}")
    print(f"Jaccard Similarity Score: {jaccard_sim:.4f}")
    print(f"MMR Score: {cos_sim:.4f}\n")  # Printing MMR Score

print(f"NDCG Score: {ndcg_score:.4f}")  # Printing NDCG Score


TF-IDF Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1:
Text: develop fiction plot
Cosine Similarity Score: 0.5508
Jaccard Similarity Score: 0.2000
MMR Score: 0.5508

Rank 2:
Text: senegalese twist
Cosine Similarity Score: 0.3957
Jaccard Similarity Score: 0.1000
MMR Score: 0.3957

Rank 3:
Text: teach science
Cosine Similarity Score: 0.3136
Jaccard Similarity Score: 0.1000
MMR Score: 0.3136

Rank 4:
Text: write engaging story
Cosine Similarity Score: 0.3048
Jaccard Similarity Score: 0.0909
MMR Score: 0.3048

Rank 5:
Text: craft child
Cosine Similarity Score: 0.2779
Jaccard Similarity Score: 0.1000
MMR Score: 0.2779

NDCG Score: 1.0000


# Modified tfidf with LDA

In [23]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize

def jaccard_similarity(query, document):
    set_query = set(query.split())
    set_doc = set(document.split())
    intersection = len(set_query & set_doc)
    union = len(set_query | set_doc)
    return intersection / union if union != 0 else 0

def mmr(query_tfidf, tfidf_matrix, lambda_param=0.5, top_k=5):
    cosine_sim = cosine_similarity(query_tfidf, tfidf_matrix).flatten()  # Keep sparse format
    
    selected_docs = []
    candidate_docs = list(range(len(cosine_sim)))
    mmr_scores = []

    for _ in range(top_k):
        if not selected_docs:
            best_idx = np.argmax(cosine_sim)
        else:
            max_sim_to_selected = np.max(cosine_similarity(tfidf_matrix[candidate_docs], tfidf_matrix[selected_docs]), axis=1)
            marginal_relevance = lambda_param * cosine_sim[candidate_docs] - (1 - lambda_param) * max_sim_to_selected
            best_idx = candidate_docs[np.argmax(marginal_relevance)]
        
        selected_docs.append(best_idx)
        candidate_docs.remove(best_idx)
        mmr_scores.append(cosine_sim[best_idx])  

    return selected_docs, mmr_scores

def compute_ndcg(relevance_scores, k):
    # Sorting the relevance scores to simulate ground truth
    sorted_relevance = sorted(relevance_scores, reverse=True)
    
    # Compute DCG
    dcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(relevance_scores[:k])])
    
    # Compute ideal DCG (IDCG) using sorted relevance
    idcg = sum([rel / np.log2(i + 2) for i, rel in enumerate(sorted_relevance[:k])])
    
    # Avoid division by zero
    ndcg = dcg / idcg if idcg != 0 else 0
    return ndcg

def modified_tfidf_retrieval(query, corpus, top_k=5, use_dim_reduction=False, n_components=50):
    # Optimized TF-IDF Vectorizer with improved preprocessing
    vectorizer = TfidfVectorizer(max_features=10000, min_df=2, max_df=0.95, sublinear_tf=True)

    # Fit and transform the corpus
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Apply dimensionality reduction (LSA) if required
    if use_dim_reduction:
        svd = TruncatedSVD(n_components=n_components, random_state=42)
        tfidf_matrix = svd.fit_transform(tfidf_matrix)
        tfidf_matrix = normalize(tfidf_matrix)  # Normalize for better similarity comparison

    # Transform query into TF-IDF vector
    query_tfidf = vectorizer.transform([query])

    # Apply dimensionality reduction to query if required
    if use_dim_reduction:
        query_tfidf = svd.transform(query_tfidf)
        query_tfidf = normalize(query_tfidf)

    # Compute MMR
    mmr_indices, mmr_scores = mmr(query_tfidf, tfidf_matrix, top_k=top_k)

    # Compute Jaccard similarity
    jaccard_scores = [jaccard_similarity(query, corpus[i]) for i in mmr_indices]

    # Compute NDCG (using the cosine similarity scores as relevance)
    ndcg_score = compute_ndcg(mmr_scores, top_k)

    # Retrieve top-k results with their respective scores
    retrieved_results = [(corpus[i], mmr_scores[j], jaccard_scores[j], mmr_scores[j]) for j, i in enumerate(mmr_indices)]

    return retrieved_results, ndcg_score

retrieved_docs, ndcg_score = modified_tfidf_retrieval(query, corpus, top_k=5, use_dim_reduction=True, n_components=50)

print(f"Modified TF-IDF Retrieval - Query: {query}")
for idx, (retrieved_text, cos_sim, jaccard_sim, mmr_score) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}:")
    print(f"Text: {retrieved_text}")
    print(f"Cosine Similarity Score: {cos_sim:.4f}")
    print(f"Jaccard Similarity Score: {jaccard_sim:.4f}")
    print(f"MMR Score: {mmr_score:.4f}")
    print("\n")

print(f"NDCG Score: {ndcg_score:.4f}")


Modified TF-IDF Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1:
Text: craft feather duster
Cosine Similarity Score: 0.8669
Jaccard Similarity Score: 0.0909
MMR Score: 0.8669


Rank 2:
Text: personal journal often full emotion deep thought snapshot daily life detail serve well decide write novel
Cosine Similarity Score: 0.4081
Jaccard Similarity Score: 0.0000
MMR Score: 0.4081


Rank 3:
Text: writing speech already challenge bother adding humor successful humor relax audience making attentive receptive
Cosine Similarity Score: 0.7823
Jaccard Similarity Score: 0.0000
MMR Score: 0.7823


Rank 4:
Text: create christmas list letter santa
Cosine Similarity Score: 0.3365
Jaccard Similarity Score: 0.0000
MMR Score: 0.3365


Rank 5:
Text: poem child fun genre many child interested poetry contains imagination creativity humor write poem child brainstorming creative idea selecting poetic form engaging young reader
Cosine Similarity Score: 0.5877
Jaccard Similarity S

# BM25

In [24]:
!pip install bm25
!pip install rank-bm25

In [25]:
import numpy as np
import warnings
from rank_bm25 import BM25Okapi
from sklearn.metrics import ndcg_score

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning, module="nltk")

def bm25_retrieval(query, corpus, top_k=5):
    # Tokenize the corpus and query
    tokenized_corpus = [doc.split() for doc in corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)

    # Get the top-k documents based on BM25 score
    sorted_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    
    retrieved_results = [(corpus[i], scores[i]) for i in sorted_idx]
    
    return retrieved_results, scores

def compute_ndcg(retrieved_docs, relevant_scores, top_k=5):
    # Create binary relevance labels for NDCG calculation (relevant or not)
    relevance = [1 if i < top_k else 0 for i in range(len(retrieved_docs))]
    
    # Compute NDCG score
    return ndcg_score([relevance], [relevant_scores[:top_k]])

# Perform retrieval using BM25
retrieved_docs, bm25_scores = bm25_retrieval(query, corpus, top_k=5)

# Compute NDCG score based on retrieved documents
ndcg_score_value = compute_ndcg(retrieved_docs, bm25_scores, top_k=5)

# Print retrieved documents and scores
print(f"Standard BM25 Retrieval - Query: {query}")
for idx, (retrieved_text, score) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}:")
    print(f"Text: {retrieved_text}")
    print(f"BM25 Score: {score:.4f}")
    print("\n")

print(f"NDCG Score: {ndcg_score_value:.4f}")


Standard BM25 Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1:
Text: develop fiction plot
BM25 Score: 22.7099


Rank 2:
Text: write science fiction
BM25 Score: 20.6905


Rank 3:
Text: throw science fiction party
BM25 Score: 19.4924


Rank 4:
Text: write story full plot twist
BM25 Score: 19.2509


Rank 5:
Text: start science fiction book club
BM25 Score: 18.4254


NDCG Score: 1.0000


# BM25 modified with query expansion

In [27]:
import numpy as np
import heapq
import warnings
from rank_bm25 import BM25Okapi
from nltk.corpus import wordnet  # For optional synonym expansion
from sklearn.metrics import ndcg_score

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Step 1: Efficient Tokenization & BM25 Indexing
def preprocess_corpus(corpus):
    return [doc.split() for doc in corpus]

# Step 2: Synonym Expansion for Queries (Optional, improves recall)
def expand_query(query):
    words = query.split()
    expanded_query = set(words)  # Use set to avoid duplicates

    for word in words:
        synonyms = wordnet.synsets(word)
        for syn in synonyms:
            for lemma in syn.lemmas():
                expanded_query.add(lemma.name().replace("_", " "))  # Add synonyms

    return " ".join(expanded_query)

# Step 3: Optimized BM25 Retrieval (Without TF-IDF)
def optimized_bm25_retrieval(query, corpus, bm25, top_k=5, use_synonyms=False):
    if use_synonyms:
        query = expand_query(query)

    query_tokens = query.split()

    # Score documents using BM25
    scores = np.array(bm25.get_scores(query_tokens))

    # Ensure top_k is not greater than available documents
    top_k = min(top_k, len(scores))

    # Efficient retrieval using heapq.nlargest()
    top_k_idx = heapq.nlargest(top_k, range(len(scores)), key=scores.__getitem__)

    # Ensure valid indices before returning results
    retrieved_results = [(corpus[i], scores[i]) for i in top_k_idx if i < len(corpus)]
    
    return retrieved_results, scores

# Step 4: Compute NDCG Score
def compute_ndcg(retrieved_docs, scores, top_k=5):
    # Create binary relevance labels for NDCG calculation (relevant or not)
    relevance = [1 if i < top_k else 0 for i in range(len(retrieved_docs))]
    
    # Compute NDCG score
    return ndcg_score([relevance], [scores[:top_k]])

# Preprocess corpus once
tokenized_corpus = preprocess_corpus(corpus)

# Initialize BM25 model
bm25 = BM25Okapi(tokenized_corpus)

# Perform retrieval with or without query expansion
use_query_expansion = True  # Change to False if you don't want synonym expansion
retrieved_docs, bm25_scores = optimized_bm25_retrieval(query, corpus, bm25, top_k=5, use_synonyms=use_query_expansion)

# Compute NDCG score based on retrieved documents
ndcg_score_value = compute_ndcg(retrieved_docs, bm25_scores, top_k=5)

# Print results
print(f"Optimized BM25 Retrieval - Query: {query}")
for idx, (retrieved_text, score) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}:")
    print(f"Text: {retrieved_text}")
    print(f"BM25 Score: {score:.4f}\n")

print(f"NDCG Score: {ndcg_score_value:.4f}")


Optimized BM25 Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1:
Text: write story full plot twist
BM25 Score: 48.8249

Rank 2:
Text: develop fiction plot
BM25 Score: 45.8941

Rank 3:
Text: writing story want write fun plot usually mean classic elusive plot twist happens write bunch twist
BM25 Score: 45.8377

Rank 4:
Text: construction disruptive environment around plot want save tree bush around plot home need take precaution construction start
BM25 Score: 44.8653

Rank 5:
Text: write story using witch plot diagram
BM25 Score: 40.6392

NDCG Score: 1.0000


# word2vec

In [ ]:
import numpy as np
import warnings
from gensim.models import Word2Vec
from nltk.translate.bleu_score import sentence_bleu

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def word2vec_retrieval(query, corpus, model, top_k=5):
    """
    Optimized Word2Vec-based document retrieval using cosine similarity.
    """
    zero_vector = np.zeros(model.vector_size, dtype=np.float32)

    # Tokenize the corpus and compute document embeddings efficiently
    tokenized_corpus = [doc.split() for doc in corpus]
    doc_vectors = np.array([
        np.mean([model.wv[word] for word in doc if word in model.wv], axis=0)
        if any(word in model.wv for word in doc) else zero_vector
        for doc in tokenized_corpus
    ], dtype=np.float32)

    # Compute query embedding
    tokenized_query = query.split()
    query_vector = np.mean(
        [model.wv[word] for word in tokenized_query if word in model.wv], axis=0
    ) if any(word in model.wv for word in tokenized_query) else zero_vector

    # Normalize for cosine similarity
    doc_vectors /= np.linalg.norm(doc_vectors, axis=1, keepdims=True) + 1e-9  # Prevent division by zero
    query_vector /= np.linalg.norm(query_vector) + 1e-9

    # Compute cosine similarity using dot product
    cosine_sim = np.dot(doc_vectors, query_vector)

    # Compute Perplexity (lower is better)
    normalized_scores = cosine_sim / np.sum(cosine_sim) if np.sum(cosine_sim) != 0 else np.ones_like(cosine_sim)
    perplexity = np.exp(-np.sum(normalized_scores * np.log(normalized_scores + 1e-9)))

    # Get top-k indices using efficient argpartition (faster than argsort)
    top_k_indices = np.argpartition(-cosine_sim, top_k)[:top_k]  # Partial sort
    top_k_indices = top_k_indices[np.argsort(-cosine_sim[top_k_indices])]  # Sort top-k

    retrieved_results = [(corpus[i], cosine_sim[i]) for i in top_k_indices]
    
    return retrieved_results, perplexity

# Step 2: BLEU Score Evaluation
def evaluate_bleu(retrieved_docs, query):
    if not retrieved_docs:
        return 0.0  # No retrieved documents, BLEU score is 0

    reference = retrieved_docs[0][0].split()  # Top retrieved document
    candidate = query.split()

    bleu_score = sentence_bleu([reference], candidate)
    return bleu_score

# Step 3: Compute Semantic Coherence Score
def semantic_coherence(retrieved_docs, model):
    """
    Computes the semantic coherence score by averaging cosine similarities among retrieved docs.
    """
    if len(retrieved_docs) < 2:
        return 0.0  # Cannot compute coherence with only one document

    vectors = [np.mean([model.wv[word] for word in doc.split() if word in model.wv], axis=0)
               for doc, _ in retrieved_docs]

    coherence_scores = []
    for i in range(len(vectors)):
        for j in range(i + 1, len(vectors)):
            sim = np.dot(vectors[i], vectors[j]) / (np.linalg.norm(vectors[i]) * np.linalg.norm(vectors[j]) + 1e-9)
            coherence_scores.append(sim)

    return np.mean(coherence_scores) if coherence_scores else 0.0


# Train Word2Vec model
model = Word2Vec([doc.split() for doc in corpus], vector_size=100, window=5, min_count=1, workers=4)

# Perform optimized Word2Vec retrieval
retrieved_docs, perplexity = word2vec_retrieval(query, corpus, model, top_k=5)

# Compute BLEU Score
bleu_score = evaluate_bleu(retrieved_docs, query)

# Compute Semantic Coherence Score
coherence_score = semantic_coherence(retrieved_docs, model)

# Print retrieved documents and scores
print(f"Optimized Word2Vec Retrieval - Query: {query}")
for idx, (retrieved_text, score) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}:")
    print(f"Text: {retrieved_text}")
    print(f"Cosine Similarity Score: {score:.4f}\n")

print(f"Perplexity: {perplexity:.4f}")
print(f"BLEU Score: {bleu_score:.4f}")
print(f"Semantic Coherence Score: {coherence_score:.4f}")


Optimized Word2Vec Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1:
Text: origami japanese art date back century origami creation range simple delightful complex astounding origami butterfly easy beginner project making perfect child activity
Cosine Similarity Score: 0.8910

Rank 2:
Text: tricking martial art tricking informal name relatively new underground alternative sport movement combining martial art gymnastics activity create aesthetic blend flip kick twist
Cosine Similarity Score: 0.8793

Rank 3:
Text: design functioning art craft studio
Cosine Similarity Score: 0.8783

Rank 4:
Text: vivid imagination dreaming story dinosaur pirate origin creativity innovation expressed everything technology science art literature
Cosine Similarity Score: 0.8774

Rank 5:
Text: mouse long considered delightful creature purpose portrayal art craft walnut mouse cute used school fundraising art project
Cosine Similarity Score: 0.8731

Perplexity: nan
BLEU Score: 0.0000

C:\Users\madhe\AppData\Local\Temp\ipykernel_11716\2703838349.py:39: RuntimeWarning: invalid value encountered in log
  perplexity = np.exp(-np.sum(normalized_scores * np.log(normalized_scores + 1e-9)))


# word2vec modified with faiss

In [20]:
!pip install faiss-cpu
import faiss

In [21]:
import numpy as np
import faiss
import warnings
from gensim.models import Word2Vec
from nltk.translate.bleu_score import sentence_bleu

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def precompute_doc_vectors(corpus, model):
    """
    Precompute document vectors and store them for FAISS retrieval.
    """
    tokenized_corpus = [doc.split() for doc in corpus]

    doc_vectors = np.array([
        np.mean([model.wv[word] for word in doc if word in model.wv] or [np.zeros(model.vector_size)], axis=0)
        for doc in tokenized_corpus
    ]).astype(np.float32)  # Ensure FAISS-compatible type

    # Replace NaN values with zero vectors (prevents FAISS errors)
    doc_vectors = np.nan_to_num(doc_vectors)

    return doc_vectors

def build_faiss_index(doc_vectors):
    """
    Build and return a FAISS index for fast retrieval.
    """
    vector_dim = doc_vectors.shape[1]
    faiss_index = faiss.IndexFlatL2(vector_dim)  # L2 distance for similarity

    # Only add non-zero vectors to FAISS
    if len(doc_vectors) > 0:
        faiss_index.add(doc_vectors)

    return faiss_index

def faiss_word2vec_retrieval(query, corpus, model, faiss_index, doc_vectors, top_k=5):
    """
    Retrieve top-k similar documents using FAISS.
    """
    tokenized_query = query.split()
    query_vector = np.mean(
        [model.wv[word] for word in tokenized_query if word in model.wv] or [np.zeros(model.vector_size)],
        axis=0
    ).astype(np.float32)

    query_vector = np.nan_to_num(query_vector).reshape(1, -1)  # Prevent NaN issues

    # Check if FAISS index is empty (avoid runtime errors)
    if faiss_index.ntotal == 0:
        print("FAISS index is empty. No documents available for retrieval.")
        return [], 0.0

    # Perform FAISS search
    distances, top_k_indices = faiss_index.search(query_vector, min(top_k, faiss_index.ntotal))

    # Ensure valid index retrieval
    retrieved_results = []
    similarity_scores = []
    for j, i in enumerate(top_k_indices[0]):
        if i < len(corpus):  # Avoid out-of-bounds indexing
            similarity_score = 1 - (distances[0][j] / (np.linalg.norm(doc_vectors[i]) + 1e-9))  # Normalize similarity
            retrieved_results.append((corpus[i], similarity_score))
            similarity_scores.append(similarity_score)

    # Compute Perplexity (lower is better)
    if similarity_scores:
        normalized_scores = np.array(similarity_scores) / np.sum(similarity_scores)
        perplexity = np.exp(-np.sum(normalized_scores * np.log(normalized_scores + 1e-9)))
    else:
        perplexity = float('inf')

    return retrieved_results, perplexity

# Step 2: BLEU Score Evaluation
def evaluate_bleu(retrieved_docs, query):
    if not retrieved_docs:
        return 0.0  # No retrieved documents, BLEU score is 0

    reference = retrieved_docs[0][0].split()  # Top retrieved document
    candidate = query.split()

    bleu_score = sentence_bleu([reference], candidate)
    return bleu_score

# Step 3: Compute Semantic Coherence Score
def semantic_coherence(retrieved_docs, model):
    """
    Computes the semantic coherence score by averaging cosine similarities among retrieved docs.
    """
    if len(retrieved_docs) < 2:
        return 0.0  # Cannot compute coherence with only one document

    vectors = [np.mean([model.wv[word] for word in doc.split() if word in model.wv], axis=0)
               for doc, _ in retrieved_docs]

    coherence_scores = []
    for i in range(len(vectors)):
        for j in range(i + 1, len(vectors)):
            sim = np.dot(vectors[i], vectors[j]) / (np.linalg.norm(vectors[i]) * np.linalg.norm(vectors[j]) + 1e-9)
            coherence_scores.append(sim)

    return np.mean(coherence_scores) if coherence_scores else 0.0


# Train Word2Vec model
model = Word2Vec([doc.split() for doc in corpus], vector_size=100, window=5, min_count=1, workers=4)

# Precompute document vectors
doc_vectors = precompute_doc_vectors(corpus, model)

# Build FAISS index
faiss_index = build_faiss_index(doc_vectors)

retrieved_docs, perplexity = faiss_word2vec_retrieval(query, corpus, model, faiss_index, doc_vectors, top_k=5)

# Compute BLEU Score
bleu_score = evaluate_bleu(retrieved_docs, query)

# Compute Semantic Coherence Score
coherence_score = semantic_coherence(retrieved_docs, model)

# Print retrieved documents and scores
print(f"FAISS Optimized Word2Vec Retrieval - Query: {query}")
for idx, (retrieved_text, score) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}:")
    print(f"Text: {retrieved_text}")
    print(f"Similarity Score: {score:.4f}\n")

print(f"Perplexity: {perplexity:.4f}")
print(f"BLEU Score: {bleu_score:.4f}")
print(f"Semantic Coherence Score: {coherence_score:.4f}")


FAISS Optimized Word2Vec Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1:
Text: tricking martial art tricking informal name relatively new underground alternative sport movement combining martial art gymnastics activity create aesthetic blend flip kick twist
Similarity Score: 0.1197

Rank 2:
Text: vivid imagination dreaming story dinosaur pirate origin creativity innovation expressed everything technology science art literature
Similarity Score: -0.0368

Rank 3:
Text: mouse long considered delightful creature purpose portrayal art craft walnut mouse cute used school fundraising art project
Similarity Score: 0.0481

Rank 4:
Text: neopoprealism style art aim combine brightness simplicity modern pop art philosophical psychological depth realism
Similarity Score: -0.1802

Rank 5:
Text: pom pom painting wonderful activity giving young child first introduction art craft unlike traditional finger painting notoriously messy disorganized painting fabric pom pom enc

C:\Users\madhe\AppData\Local\Temp\ipykernel_20304\3563761695.py:72: RuntimeWarning: invalid value encountered in log
  perplexity = np.exp(-np.sum(normalized_scores * np.log(normalized_scores + 1e-9)))


# Fasttext

In [9]:
import numpy as np
import fasttext
import fasttext.util
import nltk
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics.pairwise import cosine_similarity
import math

# Download FastText model
fasttext.util.download_model('en', if_exists='ignore')
ft_model = fasttext.load_model('cc.en.300.bin')

def fasttext_retrieval(query, corpus, top_k=5):
    query_vec = ft_model.get_sentence_vector(query)  # Convert query to vector
    similarities = []

    for i, text in enumerate(corpus):
        text_vec = ft_model.get_sentence_vector(text)  # Convert corpus sentence to vector
        similarity = np.dot(query_vec, text_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(text_vec))
        similarities.append((i, similarity))

    # Sort by similarity score (descending order) and return top_k results
    top_indices = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    return [corpus[i[0]] for i in top_indices], [similarities[i[0]][1] for i in top_indices]


# Perplexity Calculation
def calculate_perplexity(retrieved_texts, ngram_model):
    total_log_prob = 0
    total_words = 0
    
    for text in retrieved_texts:
        words = text.split()
        total_words += len(words)
        
        for i in range(len(words) - ngram_model.n + 1):
            context = tuple(words[i:i + ngram_model.n - 1])
            word = words[i + ngram_model.n - 1]
            prob = (ngram_model.ngram_counts[context][word] + 1) / (ngram_model.context_counts[context] + len(ngram_model.unigram_counts))
            total_log_prob += math.log(prob)
    
    perplexity = math.exp(-total_log_prob / total_words)
    return perplexity


# BLEU Score Calculation
def calculate_bleu(retrieved_texts, reference_texts):
    bleu_scores = []
    for retrieved, reference in zip(retrieved_texts, reference_texts):
        ref_tokens = [reference.split()]
        ret_tokens = retrieved.split()
        bleu_scores.append(sentence_bleu(ref_tokens, ret_tokens))
    
    return np.mean(bleu_scores)


retrieved_sentences, similarity_scores = fasttext_retrieval(query, corpus, top_k=5)

# Get reference summaries for evaluation
reference_summaries = df["cleaned_summary"].tolist()[:5]

# Calculate evaluation metrics
perplexity_score = calculate_perplexity(retrieved_sentences, ngram_model)
bleu_score = calculate_bleu(retrieved_sentences, reference_summaries)
mean_cosine_similarity = np.mean(similarity_scores)
max_cosine_similarity = np.max(similarity_scores)

# Print Evaluation Scores
print(f"Perplexity Score: {perplexity_score:.3f}")
print(f"BLEU Score: {bleu_score:.3f}")
print(f"Mean Cosine Similarity: {mean_cosine_similarity:.3f}")
print(f"Max Cosine Similarity: {max_cosine_similarity:.3f}")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import fasttext
import fasttext.util
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
import math
from collections import Counter

ft_model = fasttext.load_model('cc.en.300.bin')


def get_weighted_sentence_vector(sentence, model):
    words = sentence.split()
    word_counts = Counter(words)
    total_count = sum(word_counts.values())

    vectors = []
    weights = []

    for word, count in word_counts.items():
        if word in model:
            vectors.append(model.get_word_vector(word) * (count / total_count))
            weights.append(count / total_count)

    if len(vectors) == 0:
        return np.zeros(model.get_dimension())  # Return zero vector if no words found

    sentence_vector = np.sum(vectors, axis=0) / sum(weights)  # Weighted sum
    return sentence_vector / np.linalg.norm(sentence_vector)  # Normalize


def fasttext_retrieval(query, corpus, top_k=5):
    query_vec = get_weighted_sentence_vector(query, ft_model)  # Weighted FastText vector for query
    similarities = []

    for i, text in enumerate(corpus):
        text_vec = get_weighted_sentence_vector(text, ft_model)
        similarity = np.dot(query_vec, text_vec)  # Cosine similarity
        similarities.append((i, similarity))

    # Sort by similarity score (descending order) and return top_k results
    top_indices = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]
    return [corpus[i[0]] for i in top_indices], [similarities[i[0]][1] for i in top_indices]


# Perplexity Calculation (with Laplace Smoothing)
def calculate_perplexity(retrieved_texts, ngram_model, alpha=1.0):
    total_log_prob = 0
    total_words = 0

    for text in retrieved_texts:
        words = text.split()
        total_words += len(words)

        for i in range(len(words) - ngram_model.n + 1):
            context = tuple(words[i:i + ngram_model.n - 1])
            word = words[i + ngram_model.n - 1]
            prob = (ngram_model.ngram_counts[context][word] + alpha) / \
                   (ngram_model.context_counts[context] + alpha * len(ngram_model.unigram_counts))
            total_log_prob += math.log(prob)

    perplexity = math.exp(-total_log_prob / total_words)
    return perplexity


# BLEU Score Calculation (with Smoothing)
def calculate_bleu(retrieved_texts, reference_texts):
    bleu_scores = []
    smooth_fn = SmoothingFunction().method3  # Apply smoothing

    for retrieved, reference in zip(retrieved_texts, reference_texts):
        ref_tokens = [reference.split()]
        ret_tokens = retrieved.split()
        bleu_scores.append(sentence_bleu(ref_tokens, ret_tokens, smoothing_function=smooth_fn))

    return np.mean(bleu_scores)

retrieved_sentences, similarity_scores = fasttext_retrieval(query, corpus, top_k=5)

# Get reference summaries for evaluation
reference_summaries = df["cleaned_summary"].tolist()[:5]

# Calculate evaluation metrics
perplexity_score = calculate_perplexity(retrieved_sentences, ngram_model, alpha=1.0)  # Laplace smoothing
bleu_score = calculate_bleu(retrieved_sentences, reference_summaries)
mean_cosine_similarity = np.mean(similarity_scores)
max_cosine_similarity = np.max(similarity_scores)

# Print Evaluation Scores
print(f"Perplexity Score: {perplexity_score:.3f}")
print(f"BLEU Score: {bleu_score:.3f}")
print(f"Mean Cosine Similarity: {mean_cosine_similarity:.3f}")
print(f"Max Cosine Similarity: {max_cosine_similarity:.3f}")


## Glove



In [23]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load GloVe embeddings
def load_glove_embeddings(glove_file_path, embedding_dim=50):
    glove_embeddings = {}
    with open(glove_file_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]  # First word is the key
            vector = np.asarray(values[1:], dtype=np.float32)  # Remaining are vector values
            glove_embeddings[word] = vector
    return glove_embeddings

# Use the 50-dimensional GloVe file
glove_file_path = "glove.6B.50d.txt"  # Update for 50d version
embedding_dim = 50
glove_embeddings = load_glove_embeddings(glove_file_path, embedding_dim=embedding_dim)

print(f"Loaded {len(glove_embeddings)} word vectors.")

Loaded 400000 word vectors.


In [ ]:
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu

# Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

def glove_retrieval(query, corpus, glove_embeddings, embedding_dim=50, top_k=5):
    """
    Retrieve top-k similar documents using GloVe embeddings and cosine similarity.
    """
    # Tokenize the query and corpus
    query_tokens = query.split()
    tokenized_corpus = [doc.split() for doc in corpus]

    # Get GloVe vectors for the query
    query_vectors = [glove_embeddings.get(word, np.zeros(embedding_dim)) for word in query_tokens]

    # Handle case where no query words exist in the embeddings
    query_vector = np.mean(query_vectors, axis=0) if query_vectors else np.zeros(embedding_dim)

    # Ensure query_vector is 2D
    query_vector = query_vector.reshape(1, -1)

    # Get document vectors
    doc_vectors = []
    for doc in tokenized_corpus:
        vectors = [glove_embeddings.get(word, np.zeros(embedding_dim)) for word in doc]
        doc_vectors.append(np.mean(vectors, axis=0) if vectors else np.zeros(embedding_dim))

    # Convert to NumPy array
    doc_vectors = np.array(doc_vectors)

    # Compute cosine similarities
    cosine_sim = cosine_similarity(query_vector, doc_vectors)

    # Retrieve top-k documents
    similar_docs_idx = cosine_sim.argsort()[0][-top_k:][::-1]
    similar_docs_scores = cosine_sim[0][similar_docs_idx]

    retrieved_docs = [(corpus[i], similar_docs_scores[j]) for j, i in enumerate(similar_docs_idx)]

    # Compute Perplexity
    if similar_docs_scores.size > 0:
        normalized_scores = similar_docs_scores / np.sum(similar_docs_scores)
        perplexity = np.exp(-np.sum(normalized_scores * np.log(normalized_scores + 1e-9)))
    else:
        perplexity = float('inf')

    return retrieved_docs, perplexity

# BLEU Score Evaluation
def evaluate_bleu(retrieved_docs, query):
    """
    Compute BLEU score between the query and top retrieved document.
    """
    if not retrieved_docs:
        return 0.0  # No retrieved documents, BLEU score is 0

    reference = retrieved_docs[0][0].split()  # Top retrieved document
    candidate = query.split()

    return sentence_bleu([reference], candidate)

# Semantic Coherence Score
def semantic_coherence(retrieved_docs, glove_embeddings, embedding_dim=50):
    """
    Computes the semantic coherence score by averaging cosine similarities among retrieved docs.
    """
    if len(retrieved_docs) < 2:
        return 0.0  # Cannot compute coherence with only one document

    vectors = [np.mean([glove_embeddings.get(word, np.zeros(embedding_dim)) for word in doc.split()], axis=0)
               for doc, _ in retrieved_docs]

    coherence_scores = []
    for i in range(len(vectors)):
        for j in range(i + 1, len(vectors)):
            sim = np.dot(vectors[i], vectors[j]) / (np.linalg.norm(vectors[i]) * np.linalg.norm(vectors[j]) + 1e-9)
            coherence_scores.append(sim)

    return np.mean(coherence_scores) if coherence_scores else 0.0

# Retrieve Documents
retrieved_docs, perplexity = glove_retrieval(query, corpus, glove_embeddings, embedding_dim=embedding_dim, top_k=3)

# Compute BLEU Score
bleu_score = evaluate_bleu(retrieved_docs, query)

# Compute Semantic Coherence Score
coherence_score = semantic_coherence(retrieved_docs, glove_embeddings, embedding_dim=embedding_dim)

# Print retrieved documents and scores
print(f"GloVe Retrieval - Query: {query}")
for idx, (retrieved_text, score) in enumerate(retrieved_docs, 1):
    print(f"Rank {idx}: {retrieved_text} (Score: {score:.4f})")

print(f"Perplexity: {perplexity:.4f}")
print(f"BLEU Score: {bleu_score:.4f}")
print(f"Semantic Coherence Score: {coherence_score:.4f}")


GloVe Retrieval - Query: how to craft an engaging science fiction plot twist
Rank 1: writing fiction whether novel screenplay short story one challenging important task author create good character drive plot connect (Score: 0.9556)
Rank 2: writing novel creating dialogue difficult task want speech character sound natural even challenging novel set historical setting (Score: 0.9431)
Rank 3: futuristic fiction basic element story perspective plot character set apart distinctive setting like write story set future start imagining future world (Score: 0.9405)
Perplexity: 2.9999
BLEU Score: 0.0000
Semantic Coherence Score: 0.9620


# n grams

In [25]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\madhe\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [42]:
import numpy as np 
import spacy
import random
import nltk
import warnings
from collections import defaultdict, Counter
from nltk.translate.bleu_score import sentence_bleu

# Suppress warnings
warnings.filterwarnings("ignore")

# Load spacy NLP model for sentence segmentation
nlp = spacy.load("en_core_web_sm")

# Step 1: Retrieve top-k similar queries using BM25
def bm25_retrieval(query, top_k=5):
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return [corpus[i] for i in top_indices]

# Step 2: Find related words using GloVe
def find_related_words(query, top_n=5):
    query_tokens = query.split()
    related_words = []

    for word in query_tokens:
        if word in glove_embeddings:
            word_vec = glove_embeddings[word].reshape(1, -1)
            similarities = {
                candidate: np.dot(word_vec, vec.reshape(-1, 1))[0][0]
                for candidate, vec in glove_embeddings.items()
            }
            top_related = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:top_n]
            related_words.extend([w[0] for w in top_related])

    return list(set(related_words))

# Step 3: Build an improved 4-Gram Model with Kneser-Ney Smoothing
class NGramModel:
    def __init__(self, n=4):
        self.n = n
        self.ngram_counts = defaultdict(Counter)
        self.context_counts = defaultdict(int)
        self.unigram_counts = Counter()
        self.total_words = 0

    def train(self, text_corpus):
        for sentence in text_corpus:
            tokens = ["<s>"] * (self.n - 1) + sentence.split() + ["</s>"]
            for i in range(len(tokens) - self.n + 1):
                context = tuple(tokens[i:i + self.n - 1])
                word = tokens[i + self.n - 1]
                self.ngram_counts[context][word] += 1
                self.context_counts[context] += 1

            for token in tokens:
                self.unigram_counts[token] += 1
                self.total_words += 1

    def generate(self, seed_text, max_words=50, discount=0.75, repetition_penalty=0.5):
        seed_tokens = seed_text.split()
        seed_tokens = ["<s>"] * (self.n - 1) + seed_tokens

        generated_text = list(seed_tokens[-(self.n - 1):])
        word_usage = defaultdict(int)  

        for _ in range(max_words):
            context = tuple(generated_text[-(self.n - 1):])
            word_probs = self.ngram_counts.get(context, {})

            if not word_probs:
                if len(context) > 1:
                    context = context[1:]
                    word_probs = self.ngram_counts.get(context, {})

            if not word_probs:
                word_probs = self.unigram_counts  

            discounted_probs = {}
            total_discounted = 0
            for word, count in word_probs.items():
                context_count = self.context_counts.get(context, 1)  
                if count > discount:
                    discounted_probs[word] = (count - discount) / context_count
                    total_discounted += discounted_probs[word]

            lambda_weight = discount / context_count if context_count > 0 else 0
            lower_order_probs = self.unigram_counts

            kn_probs = {}
            for word in discounted_probs:
                penalty = repetition_penalty ** word_usage[word]
                kn_probs[word] = (discounted_probs[word] + lambda_weight * lower_order_probs.get(word, 1e-6)) * penalty

            total_prob = sum(kn_probs.values())
            if total_prob == 0:
                kn_probs = lower_order_probs  

            words, probabilities = zip(*kn_probs.items())
            prob_sum = sum(probabilities)
            probabilities = np.array(probabilities) / prob_sum if prob_sum > 0 else [1 / len(words)] * len(words)

            next_word = np.random.choice(words, p=probabilities)

            if next_word in ["</s>", "<s>"]:
                continue

            word_pos = nltk.pos_tag([next_word])[0][1]
            prev_pos = nltk.pos_tag([generated_text[-1]])[0][1] if len(generated_text) > 1 else None

            if prev_pos in ["JJ", "RB"] and word_pos in ["JJ", "RB"]:
                continue

            generated_text.append(next_word)
            word_usage[next_word] += 1  

            if next_word in [".", "!", "?"]:
                generated_text.append("\n")

        final_output = " ".join(generated_text[(self.n - 1):])
        final_output = final_output.replace(" <s>", "").replace("<s> ", "").replace("<s>", "").replace(" </s>", "").replace("</s> ", "").strip()

        return final_output

    def calculate_perplexity(self, test_sentence):
        tokens = test_sentence.split()
        test_tokens = ["<s>"] * (self.n - 1) + tokens + ["</s>"]
        log_prob_sum = 0
        N = len(test_tokens) - self.n + 1

        for i in range(N):
            context = tuple(test_tokens[i:i + self.n - 1])
            word = test_tokens[i + self.n - 1]
            word_prob = (self.ngram_counts[context][word] + 1) / (self.context_counts[context] + len(self.unigram_counts))
            log_prob_sum += np.log(word_prob)

        perplexity = np.exp(-log_prob_sum / N)
        return perplexity

# Step 4: Sentence Structuring
def structure_sentences(text):
    doc = nlp(text)
    structured_text = " ".join([sent.text.capitalize() for sent in doc.sents])
    return structured_text

# Step 5: BLEU Score Calculation
def calculate_bleu(reference_texts, generated_text):
    references = [ref.split() for ref in reference_texts]
    candidate = generated_text.split()
    return sentence_bleu(references, candidate)

# Step 6: Main function to generate structured content and evaluate it
def generate_content(query):
    print(f"Query: {query}")

    retrieved_prompts = bm25_retrieval(query, top_k=5)
    related_words = find_related_words(query, top_n=5)

    ngram_model = NGramModel(n=4)
    ngram_model.train(retrieved_prompts)

    generated_text = ngram_model.generate(" ".join(related_words), max_words=50)
    structured_output = structure_sentences(generated_text)

    # Evaluate Perplexity
    perplexity = ngram_model.calculate_perplexity(structured_output)
    print(f"Perplexity: {perplexity:.3f}")

    # Evaluate BLEU Score
    bleu_score = calculate_bleu(retrieved_prompts, structured_output)
    print(f"BLEU Score: {bleu_score:.3f}")

    print(f"Generated Structured Text: {structured_output}")
    return structured_output, perplexity, bleu_score

# Example query
query = "how to lead a healthy life"
generated_text, perplexity, bleu_score = generate_content(query)


Query: how to lead a healthy life
Perplexity: 18.234
BLEU Score: 0.000
Generated Structured Text: Celebrity lead life possible life happy lead healthy mind healthy best healthy life possible stress happy step long


# cia 2

In [18]:
df.columns

Index(['text', 'summary', 'cleaned_text', 'cleaned_summary'], dtype='object')

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Combine text and summary for a global vocabulary
all_texts = df['cleaned_text'].tolist()

# Tokenizer initialization and fitting
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)

# Vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
print("Vocabulary Size:", vocab_size)


Vocabulary Size: 47921


# GRU

In [20]:
import tensorflow as tf
from keras import backend as K

# Clear TensorFlow session
K.clear_session()
tf.compat.v1.reset_default_graph()

In [37]:
del model  # Delete the previous model to avoid reinitialization issues
K.clear_session()  # Reset Keras session

In [21]:
!pip install textblob

In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from textblob import TextBlob  # For grammar correction


In [40]:
#tf.keras.mixed_precision.set_global_policy('float16')  # Enable Mixed Precision Training

In [50]:
train_texts, test_texts = train_test_split(df['cleaned_text'], test_size=0.2, random_state=42)

print("Train size:", len(train_texts))
print("Test size:", len(test_texts))


Train size: 102834
Test size: 25709


In [51]:
VOCAB_SIZE = 20000  # Reduced vocab size to speed up embedding lookup
MAX_TEXT_LENGTH = 300  # Reduced sequence length to lower memory usage

tokenizer_text = Tokenizer(num_words=VOCAB_SIZE, oov_token="<UNK>")
tokenizer_text.fit_on_texts(train_texts)

train_text_sequences = tokenizer_text.texts_to_sequences(train_texts)
train_text_padded = pad_sequences(train_text_sequences, maxlen=MAX_TEXT_LENGTH, padding='post', truncating='post')


In [52]:
embedding_dim = 50  # Ensure this matches GloVe vectors
embedding_matrix = np.zeros((VOCAB_SIZE, embedding_dim))  # Shape: (20000, 50)

word_index = tokenizer_text.word_index

for word, i in word_index.items():
    if i < VOCAB_SIZE:  # Ensure index is within range
        vector = glove_embeddings.get(word)
        if vector is not None and len(vector) == embedding_dim:  # Ensure correct shape
            embedding_matrix[i] = vector  # Assign only if shape matches


In [53]:
embedding_matrix = np.zeros((VOCAB_SIZE, embedding_dim))
word_index = tokenizer_text.word_index

for word, i in word_index.items():
    if i < VOCAB_SIZE:
        vector = glove_embeddings.get(word)
        if vector is not None:
            embedding_matrix[i] = vector

# Encoder
encoder_inputs = tf.keras.layers.Input(shape=(MAX_TEXT_LENGTH,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, embedding_dim, 
                                             weights=[embedding_matrix], 
                                             input_length=MAX_TEXT_LENGTH, 
                                             trainable=False)(encoder_inputs)
encoder_gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(192, return_state=True))
encoder_outputs, forward_h, backward_h = encoder_gru(encoder_embedding)

state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])  # Final encoder state

# Decoder
decoder_inputs = tf.keras.layers.Input(shape=(MAX_TEXT_LENGTH,))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, embedding_dim, input_length=MAX_TEXT_LENGTH)(decoder_inputs)
decoder_gru = tf.keras.layers.GRU(384, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_embedding, initial_state=[state_h])

decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE, activation='softmax')
outputs = decoder_dense(decoder_outputs)

# Model Compilation
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), loss='sparse_categorical_crossentropy')

model.summary()


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 300, 50)              1000000   ['input_9[0][0]']             
                                                                                                  
 input_10 (InputLayer)       [(None, 300)]                0         []                            
                                                                                                  
 bidirectional_2 (Bidirecti  [(None, 384),                281088    ['embedding_4[0][0]']         
 onal)                        (None, 192),                                                  

In [27]:
# Ensure decoder input and target are properly aligned
decoder_input_data = train_text_padded[:, :-1]  # Remove last token
decoder_target_data = train_text_padded[:, 1:]  # Shift left

# Ensure correct shape
decoder_input_data = np.pad(decoder_input_data, ((0, 0), (0, 1)), mode='constant')
decoder_target_data = np.pad(decoder_target_data, ((0, 0), (0, 1)), mode='constant')

print("Encoder Input Shape:", train_text_padded.shape)  # (batch, 300)
print("Decoder Input Shape:", decoder_input_data.shape)  # (batch, 300)
print("Decoder Target Shape:", decoder_target_data.shape)  # (batch, 300)


Encoder Input Shape: (102834, 300)
Decoder Input Shape: (102834, 300)
Decoder Target Shape: (102834, 300)


In [47]:
import tensorflow as tf

# Define validation size (10% of training data)
val_size = int(0.1 * len(train_text_padded))  

# Create tf.data.Dataset with encoder and decoder inputs
train_dataset = tf.data.Dataset.from_tensor_slices((
    (train_text_padded[:-val_size], decoder_input_data[:-val_size]), 
    decoder_target_data[:-val_size]
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    (train_text_padded[-val_size:], decoder_input_data[-val_size:]), 
    decoder_target_data[-val_size:]
))

# Define batch size
BATCH_SIZE = 128

# Batch and prefetch datasets
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

# Define early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[early_stopping]  # Optional, but recommended
)


Epoch 1/10
724/724 [==============================] - 7753s 11s/step - loss: 0.5310 - val_loss: 0.4092
Epoch 2/10
724/724 [==============================] - 7762s 11s/step - loss: 0.3719 - val_loss: 0.3459
Epoch 3/10
724/724 [==============================] - 7900s 11s/step - loss: 0.3107 - val_loss: 0.2905
Epoch 4/10
724/724 [==============================] - 7845s 11s/step - loss: 0.2487 - val_loss: 0.2417
Epoch 5/10
724/724 [==============================] - 7863s 11s/step - loss: 0.1963 - val_loss: 0.2027
Epoch 6/10
724/724 [==============================] - 7865s 11s/step - loss: 0.1560 - val_loss: 0.1812
Epoch 7/10
724/724 [==============================] - 7903s 11s/step - loss: 0.1784 - val_loss: 0.1720
Epoch 8/10
724/724 [==============================] - 7878s 11s/step - loss: 0.1180 - val_loss: 0.1504
Epoch 9/10
724/724 [==============================] - 7925s 11s/step - loss: 0.0950 - val_loss: 0.1380
Epoch 10/10
724/724 [==============================] - 7876s 11s/step - l

In [54]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

VOCAB_SIZE = 20000
tokenizer_text = Tokenizer(num_words=VOCAB_SIZE, oov_token="<UNK>")
tokenizer_text.fit_on_texts(train_texts)  # Ensure train_texts is available

# Save the tokenizer to avoid this issue in the future
with open("tokenizer_text.pkl", "wb") as handle:
    pickle.dump(tokenizer_text, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Tokenizer recreated and saved successfully!")


Tokenizer recreated and saved successfully!


In [48]:
model.save('gru_generator.h5')

c:\Users\madhe\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
model.save('gru_generator_model.keras')

In [50]:
import numpy as np

# Convert test texts to sequences and pad
test_text_sequences = tokenizer_text.texts_to_sequences(test_texts)
test_text_padded = pad_sequences(test_text_sequences, maxlen=MAX_TEXT_LENGTH, padding='post', truncating='post')

# Ensure decoder inputs and targets
test_decoder_inputs = test_text_padded[:, :-1]  # Shifted input
test_decoder_targets = test_text_padded[:, 1:]  # Shifted target

# Evaluate model
test_loss = model.evaluate((test_text_padded, test_decoder_inputs), test_decoder_targets)
perplexity = np.exp(test_loss)  # Compute perplexity

print("Test Loss:", test_loss)
print("Perplexity (PPL):", perplexity)


804/804 [==============================] - 1135s 1s/step - loss: 0.1283
Test Loss: 0.12834285199642181
Perplexity (PPL): 1.136942738970186


In [55]:
with open("tokenizer_text.pkl", "rb") as handle:
    tokenizer_text = pickle.load(handle)
print("Tokenizer loaded successfully!")

Tokenizer loaded successfully!


In [56]:
from tensorflow.keras.models import load_model

model = load_model("gru_generator_model.keras")  

In [60]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from textblob import TextBlob
import pickle

# Load trained model
model = tf.keras.models.load_model("gru_generator_model.keras")

# Load the saved tokenizer
with open("tokenizer_text.pkl", "rb") as handle:
    tokenizer_text = pickle.load(handle)

VOCAB_SIZE = 20000  
MAX_TEXT_LENGTH = 300  

def generate_response(user_query):
    # Check tokenization
    input_seq = tokenizer_text.texts_to_sequences([user_query])
    print("Tokenized Input:", input_seq)  # Debug: Check tokenized query

    input_padded = pad_sequences(input_seq, maxlen=MAX_TEXT_LENGTH, padding='post', truncating='post')
    print("Padded Input:", input_padded)  # Debug: Check padded input

    # Initialize decoder input
    decoder_input = np.zeros((1, MAX_TEXT_LENGTH))  
    decoder_input[0, 0] = tokenizer_text.word_index.get("<START>", 1)  

    generated_text = []
    
    for i in range(MAX_TEXT_LENGTH - 1):
        predictions = model.predict([input_padded, decoder_input], verbose=0)
        next_word_index = np.argmax(predictions[0, i])

        if next_word_index == tokenizer_text.word_index.get("<END>", 2):  
            break

        next_word = tokenizer_text.index_word.get(next_word_index, "<UNK>")
        print(f"Step {i}: Predicted Token Index = {next_word_index}, Word = {next_word}")  # Debugging output
        
        generated_text.append(next_word)

        if i + 1 < MAX_TEXT_LENGTH:
            decoder_input[0, i + 1] = next_word_index

    generated_sentence = " ".join(generated_text)
    
    # Grammar correction
    corrected_sentence = str(TextBlob(generated_sentence).correct())

    return corrected_sentence

# Example usage
user_query = "how to craft an engaging science fiction plot twist"
response = generate_response(user_query)
print("Generated Response:", response)


Tokenized Input: [[1, 1, 590, 1, 2718, 1202, 3205, 3352, 1490]]
Padded Input: [[   1    1  590    1 2718 1202 3205 3352 1490    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0  

In [42]:
model.input

[<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'input_7')>,
 <KerasTensor: shape=(None, None) dtype=float32 (created by layer 'input_8')>]